Last updated by Developer on 2024-06-13.
Writeup: https://www.notion.so/Log-Usage-for-Product-aed87e70670941eca59a532f4e9f3283.

In [ ]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/user/ficc/ficc/mitas_creds.json'

In [ ]:
import sys
from datetime import datetime
from pytz import timezone

__file__ = os.path.abspath('update_usage_logs_from_gcp_storage.ipynb')    # in a Jupyter Notebook, the `__file__` variable is not automatically defined because notebooks do not run as standard Python scripts
cloud_function_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..', 'cloud_functions', 'update_usage_logs_from_gcp_storage'))    # get the directory containing the 'cloud_functions/update_usage_logs_from_gcp_storage/' package
sys.path.append(cloud_function_dir)    # add the directory to sys.path
print('NOTE: this file must be run from the `notebooks/update_usage_logs_from_gcp_storage/` directory')

from main import get_all_usage_log_filenames, combine_usage_log_filenames_into_list_of_dicts, upload_usage_dicts_to_bigquery
from auxiliary_variables import YEAR_MONTH_DAY, GCP_BUCKET_NAME
from gcp_storage_functions import delete_list_of_filenames

In [ ]:
USE_TQDM = True

Run the code in `hello_http(...)`, using a batch size of 3000. Too large of a batch does not allow us to monitor progress and a failure would cause a large restart.

In [ ]:
BATCH_SIZE = 3000

Modify `current_date_as_string` to be represent the date for which there are log files in the `server_logging` Google Cloud bucket that are to be transferred to the BigQuery table.

In [ ]:
current_date_as_string = '2024-06-07'    # datetime.now(timezone('US/Eastern')).strftime(YEAR_MONTH_DAY)
print(f'Inspecting folder with name: {current_date_as_string}')
all_usage_log_filenames = get_all_usage_log_filenames(current_date_as_string)
for start_idx in range(0, len(all_usage_log_filenames), BATCH_SIZE):
    filenames = all_usage_log_filenames[start_idx : start_idx + BATCH_SIZE]
    usage_log_list_of_dicts_for_filenames = combine_usage_log_filenames_into_list_of_dicts(filenames, USE_TQDM)
    upload_usage_dicts_to_bigquery(usage_log_list_of_dicts_for_filenames)
    delete_list_of_filenames(GCP_BUCKET_NAME, filenames, USE_TQDM)